# Brighway activities

__Difficulty:__ Basic :)

In [51]:
import bw2data
from bw2data.backends import Activity
from bw2data.backends.proxies import Exchanges

In [52]:
# Change this to your local setup
BW_EXAMPLE_PROJECT = "ecoinvent_391"

In [53]:
bw2data.projects.set_current(BW_EXAMPLE_PROJECT)
# which databases does the project have
bw2data.databases

In [54]:
# Change this to the ecoinvent or other database that has some technosphere activities
BW_EXAMPLE_DB = "ecoinvent_391_cutoff"

In [55]:
from bw2data.backends import SQLiteBackend

db: SQLiteBackend = bw2data.Database(BW_EXAMPLE_DB)
db, f"Activities: {len(db)}"

In [56]:
activity: Activity = db.random()
activity, type(activity)

In [28]:
def print_exchanges(exchange_list: Exchanges):
    for exchange in exchange_list:
        print(exchange)
        print()
    print()

In [30]:
# biosphere inputs
biosphere_exchanges: Exchanges = activity.biosphere()
print(f"*** biosphere exchanges ({len(biosphere_exchanges)})")
print_exchanges(biosphere_exchanges)

# technosphere inputs
technosphere_exchanges = activity.technosphere()
print(f"*** technosphere exchanges ({len(technosphere_exchanges)})")
print_exchanges(technosphere_exchanges)

# technosphere outputs
upstream_exchanges: Exchanges = activity.exchanges()
# exactly the same as: activity.consumers()
print(f"*** upstream exchanges ({len(upstream_exchanges)})")
print_exchanges(upstream_exchanges)

# product
production_exchange: Exchanges = activity.production()
# exactly the same as: activity.producers
print(f"*** production exchange (generally (always) 1)")
print(list(production_exchange)[0])

In [31]:
# all inputs at once this includes the biosphere, technosphere and the production
all_inputs: Exchanges = activity.exchanges()
# exactly the same as activity.edges()
print(f"*** all inputs ({len(list(all_inputs))})")
for exchange in upstream_exchanges:
    print(exchange)
    print()

In [34]:
first_input = next(iter(all_inputs))
print(first_input.input)
print(first_input.output)

In [35]:
second_input = list(all_inputs)[1]
print(second_input.input)
print(second_input.output)

In [62]:

def collect_background_activities(activity: Activity, levels: int = 1) -> list[set[Activity]]:
    """
    
    :param activity: 
    :param levels: pass -1 for search until no new activities are added
    :return: 
    """
    background_system: list[set[Activity]] = [{activity}]
    all_activities: set[Activity] = {activity}
    current_level = 0
    # use a while loop, because passing levels: -1 means it goes on until there is nothing new to add
    while True:
        print(f"level: {current_level}")
        current_level_activities = background_system[current_level]
        next_level_activities: set[Activity] = set()
        for _activity in current_level_activities:
            technosphere_input_exchanges: Exchanges = _activity.exchanges()
            for input_exchange in technosphere_input_exchanges:
                if input_exchange.input not in all_activities:
                    next_level_activities.add(input_exchange.input)
        print(f"Adding {len(next_level_activities)} activities")
        background_system.append(next_level_activities)
        if len(next_level_activities) > 0:
            all_activities.update(next_level_activities)
        # end the loop, if no new activities have been added
        if not next_level_activities:
            break
        if levels != -1 and current_level == levels:
            break
        current_level += 1
    return background_system


collect_background_activities(activity, -1)